# Retrieving SMAP data

Data Product:
* Level 3 - daily gridded, 9km enhanced

Documentation: https://nsidc.org/sites/default/files/spl3smp_e-v005-userguide.pdf

## Setup

In [ ]:
import earthaccess
import pandas
import h5py
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
earthaccess.login()

## Retrieve

In [ ]:
collection = "SPL3SMP_E"   # daily, 9 km enhanced soil moisture
# bbox = [37, -111, 46, -95]   # [south, west, north, east]
# [west_lon, south_lat, east_lon, north_lat]
bbox = (-111, 37, -95, 46)
time_range = ("2023-01-01", "2023-01-02")

granules = earthaccess.search_data(
    short_name=collection,
    bounding_box=bbox,
    temporal=time_range
)

print(f"Found {len(granules)} granules")

In [ ]:
files = earthaccess.download(granules, "../data/SMAP")

print("Downloaded files:")
for f in files:
    print(f)

## Read and Explore

In [ ]:
fname = files[1]
print(f"Reading file: {fname}")
time_info = {}


# Get Meta
with h5py.File(fname, "r") as f:
    print(list(f.keys())) 
    meta = f["Metadata"]
    print("Metadata keys:", list(meta.keys()))


In [ ]:
# Get Observations

var_names = ["longitude", "latitude", "soil_moisture", "soil_moisture_error", "soil_moisture_dca"]

with h5py.File(fname, "r") as f:
    # print(list(f.keys()))              
    # meta = f["Metadata"]
    grp = f["Soil_Moisture_Retrieval_Data_AM"]
    
    # read one variable to get shape
    sample = grp[var_names[0]][:]
    ny, nx = sample.shape
    # allocate 3D array: (variables, y, x)
    sm1 = np.zeros((len(var_names), ny, nx), dtype=sample.dtype)
    # load each variable into the stack
    for i, name in enumerate(var_names):
        sm1[i] = grp[name][:]

In [ ]:
sm1.shape

In [ ]:
sm1[sm1 == -9999.0] = np.nan

In [ ]:
# Subset
west, south, east, north = bbox

lon_idx = var_names.index("longitude")
lat_idx = var_names.index("latitude")
lon = sm1[lon_idx]
lat = sm1[lat_idx]

mask = (
    (lon >= west) & (lon <= east) &
    (lat >= south) & (lat <= north)
)

# find bounding rows/cols (tight cropping)
rows = np.where(mask.any(axis=1))[0]
cols = np.where(mask.any(axis=0))[0]

row_min, row_max = rows.min(), rows.max()
col_min, col_max = cols.min(), cols.max()

# subset every variable automatically
sm1_sub = sm1[:, row_min:row_max+1, col_min:col_max+1]

print("Original shape:", sm1.shape)
print("Subset shape:", sm1_sub.shape)

## Visualize

In [ ]:
lon_idx = var_names.index("longitude")
lat_idx = var_names.index("latitude")
sm_idx  = var_names.index("soil_moisture")

lon = sm1_sub[lon_idx]     
lat = sm1_sub[lat_idx]
soil = sm1_sub[sm_idx]

lon_min = np.nanmin(lon)
lon_max = np.nanmax(lon)
lat_min = np.nanmin(lat)
lat_max = np.nanmax(lat)

extent = [lon_min, lon_max, lat_min, lat_max]

plt.figure(figsize=(8, 6))
im = plt.imshow(
    soil,
    origin="upper",         # flip if needed
    extent=extent,
    aspect="auto"
)
plt.colorbar(im, label="Soil Moisture (m³/m³)")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.title("SMAP Soil Moisture (AM)")
plt.show()

In [ ]:
lon_idx = var_names.index("longitude")
lat_idx = var_names.index("latitude")
sm_idx  = var_names.index("soil_moisture")

lon = sm1[lon_idx]     
lat = sm1[lat_idx]
soil = sm1[sm_idx]

lon_min = np.nanmin(lon)
lon_max = np.nanmax(lon)
lat_min = np.nanmin(lat)
lat_max = np.nanmax(lat)

extent = [lon_min, lon_max, lat_min, lat_max]

plt.figure(figsize=(8, 6))
im = plt.imshow(
    soil,
    origin="upper",         # flip if needed
    extent=extent,
    aspect="auto"
)
plt.colorbar(im, label="Soil Moisture (m³/m³)")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.title("SMAP L3_e Soil Moisture (2023-01-01 AM)")
plt.savefig("../outputs/SMAP_swaths.png")
plt.show()